In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from weather_google_apis import g_key

In [5]:
# Output File (CSV)
weather_csv = pd.read_csv("weatherpy.csv")
print(weather_csv)

     Unnamed: 0         City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
0             0    ostrovnoy  68.05   39.51     37.50        72           2   
1             1    mahebourg -20.41   57.70     78.80        74          75   
2             2      qaanaaq  77.48  -69.36     22.20        73          25   
3             3       zhuhai  40.71  112.04     44.60        39           0   
4             4    cape town -33.93   18.42     55.99        76          20   
..          ...          ...    ...     ...       ...       ...         ...   
531         543       yabrud  33.97   36.66     80.60        32           0   
532         544      paraiso  24.01 -104.61     84.20         4           5   
533         545      veraval  49.65    0.71     61.00        62           0   
534         546  novyy urgal  51.07  132.56     49.74        93         100   
535         547      tongren  27.72  109.18     55.86        64           4   

     Wind Speed Country        Date  
0          7.

In [6]:
#inplace = true overwrites your current dataframe

weather_csv.drop(columns = "Unnamed: 0", inplace = True)
weather_csv.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ostrovnoy,68.05,39.51,37.50,72,2,7.16,RU,1558378754
1,mahebourg,-20.41,57.70,78.80,74,75,11.41,MU,1558378503
2,qaanaaq,77.48,-69.36,22.20,73,25,2.37,GL,1558378755
3,zhuhai,40.71,112.04,44.60,39,0,4.47,CN,1558378755
4,cape town,-33.93,18.42,55.99,76,20,8.05,ZA,1558378755


In [7]:
gmaps.configure(api_key = g_key)
locations = weather_csv[["Lat", "Lng"]]
weight = weather_csv["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig



#get just the map already zoomed in

Figure(layout=FigureLayout(height='420px'))

In [8]:
#vacation parameters (70-80 degrees, windspeed less than 10, 0 cloudiness)

# is there a better way to get this into one step???

df1 = weather_csv.loc[weather_csv['Max Temp'] > 70]
df2 = df1.loc[weather_csv['Max Temp'] < 80]
df3 = df2.loc[weather_csv['Wind Speed'] < 10]
vacation = df3.loc[weather_csv['Cloudiness']  == 0]
vaca_df = vacation.dropna()
vaca_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,ponta do sol,-20.63,-46.00,73.68,57,0,5.32,BR,1558378767
87,cidreira,-30.17,-50.22,76.56,66,0,5.50,BR,1558378770
136,morondava,-20.30,44.28,79.08,69,0,9.06,MG,1558378780
137,vaini,15.34,74.49,78.90,88,0,1.59,IN,1558378780
173,nador,35.17,-2.93,75.20,40,0,6.93,MA,1558378499
289,mogok,22.92,96.51,72.24,51,0,2.77,MM,1558378844
432,birjand,32.86,59.22,71.60,26,0,9.17,IR,1558378880


In [9]:
# #viani did not have a city name so kept getting errors on the for loop. Dropping this. 

deleted_row = vaca_df[vaca_df["City"] == 'vaini'].index
hotel_df = vaca_df.drop(deleted_row)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,ponta do sol,-20.63,-46.00,73.68,57,0,5.32,BR,1558378767
87,cidreira,-30.17,-50.22,76.56,66,0,5.50,BR,1558378770
136,morondava,-20.30,44.28,79.08,69,0,9.06,MG,1558378780
173,nador,35.17,-2.93,75.20,40,0,6.93,MA,1558378499
289,mogok,22.92,96.51,72.24,51,0,2.77,MM,1558378844
432,birjand,32.86,59.22,71.60,26,0,9.17,IR,1558378880


In [10]:
names = []

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    coordinates = f"{lat},{lng}"
    #print(row['Lat'], row['Lng'])
    target_radius = 5000
    keyword = 'lodging'


# set up a parameters dictionary
    params = {
        "location" : coordinates,
        "radius": target_radius,
        "keyword": keyword,
        "key": g_key
        }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)

# convert response to json
    places_data = response.json()

# Print the json (pretty printed)

    try:
        output = places_data["results"][0]['name']
        print(output)
    except:
        print("Error")
        
    names.append(output)

hotel_df['Hotel Name'] = names


Pousada Carneirinho
Hotel Castelo
Palissandre Cote Ouest resort & SPA
Hôtel Mercure Rif Nador
Mount Mogok Hotel
مجتمع آریا


In [11]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
69,ponta do sol,-20.63,-46.00,73.68,57,0,5.32,BR,1558378767,Pousada Carneirinho
87,cidreira,-30.17,-50.22,76.56,66,0,5.50,BR,1558378770,Hotel Castelo
136,morondava,-20.30,44.28,79.08,69,0,9.06,MG,1558378780,Palissandre Cote Ouest resort & SPA
173,nador,35.17,-2.93,75.20,40,0,6.93,MA,1558378499,Hôtel Mercure Rif Nador
289,mogok,22.92,96.51,72.24,51,0,2.77,MM,1558378844,Mount Mogok Hotel
432,birjand,32.86,59.22,71.60,26,0,9.17,IR,1558378880,مجتمع آریا


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Column
    #using loc df.at, provide column name, value name

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))